# Module 01 — Mathematical & Programming Foundations## 01-03: Pandas for Tabular Data**Objective:** Master the Pandas library for exploratory data analysis (EDA),data cleaning, feature engineering, and preprocessing — the essential pipelinebefore any ML model training.**Prerequisites:** 01-01 (Python, NumPy & Tensor Speed), 01-02 (Advanced NumPy & PyTorch Operations)

---## Part 0 — Setup & PrerequisitesIn notebooks 01-01 and 01-02, we worked with NumPy arrays and PyTorch tensors —dense, homogeneous numerical data. Real-world ML data, however, is messy: ithas missing values, mixed types (numbers, strings, dates), and requires carefulpreprocessing before it can be fed to any model.**Pandas** is the standard tool for this. It provides:- `DataFrame` — a 2D table with labeled columns of potentially different types- `Series` — a single column with an index- Rich I/O (CSV, JSON, SQL, Excel) and powerful group-by/merge/pivot operationsWe will cover:- DataFrame creation, indexing, and selection- Exploratory data analysis (EDA) workflow- Handling missing values (detection, imputation, deletion)- Categorical encoding (label, one-hot, ordinal, target encoding)- Feature engineering (transformations, binning, interactions)- Merging, grouping, and pivoting- Converting between Pandas, NumPy, and PyTorchWe'll use the **California Housing** dataset from sklearn — a real regressiondataset with 20,640 samples and 8 numerical features.**Prerequisites:** 01-01 (Python, NumPy & Tensor Speed), 01-02 (Advanced NumPy & PyTorch Operations)

In [ ]:
# ── Imports ──────────────────────────────────────────────────────────────────
import sys
import time
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

print(f'Python: {sys.version.split()[0]}')
print(f'NumPy: {np.__version__}')
print(f'Pandas: {pd.__version__}')
print(f'PyTorch: {torch.__version__}')
if torch.cuda.is_available():
    print(f'CUDA: {torch.version.cuda}')
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
# ── Reproducibility ──────────────────────────────────────────────────────────
import random

SEED = 1103
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [ ]:
# ── Configuration ────────────────────────────────────────────────────────────
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# Display settings for Pandas
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 15)
pd.set_option('display.precision', 4)

### Data Loading & EDAWe load the California Housing dataset, which predicts median house valuefrom 8 features: median income, house age, average rooms, etc. Each rowrepresents a census block group (~600-3,000 people).

In [ ]:
# Load California Housing dataset
housing = fetch_california_housing(as_frame=True)
df = housing.frame  # Full DataFrame including target

print(f'Dataset shape: {df.shape}')
print(f'Columns: {list(df.columns)}')
print(f'Target: MedHouseVal (median house value in $100K)')
print()
df.head()

In [ ]:
# Quick EDA: types, missing values, basic statistics
print('=== Data Types ===')
print(df.dtypes)
print()

print('=== Missing Values ===')
print(df.isnull().sum())
print()

print('=== Descriptive Statistics ===')
df.describe()

---## Part 1 — Pandas Fundamentals from ScratchBefore using Pandas' high-level functions, let's understand the fundamentaldata structures and operations by building them up step by step.

### 1.1 DataFrame Creation: Multiple Ways to Build TablesA DataFrame can be created from dictionaries, lists, NumPy arrays, or loadedfrom files. Understanding these constructors is essential because data arrivesin many formats.

In [ ]:
def demonstrate_dataframe_creation() -> None:
    """Show multiple ways to create DataFrames."""
    # Method 1: From a dictionary (most common)
    dict_df = pd.DataFrame({
        'name': ['Alice', 'Bob', 'Charlie', 'Diana'],
        'age': [25, 30, 35, 28],
        'score': [92.5, 88.0, 95.3, 91.7],
        'passed': [True, True, True, True],
    })
    print('From dictionary:')
    print(dict_df)
    print(f'  dtypes: {dict(dict_df.dtypes)}')
    print()

    # Method 2: From a NumPy array
    np_data = np.random.randn(5, 3)
    np_df = pd.DataFrame(np_data, columns=['feat_a', 'feat_b', 'feat_c'])
    print('From NumPy array:')
    print(np_df)
    print()

    # Method 3: From a list of dictionaries (common with JSON/API data)
    records = [
        {'model': 'LinearReg', 'rmse': 0.85, 'r2': 0.72},
        {'model': 'RandomForest', 'rmse': 0.62, 'r2': 0.87},
        {'model': 'XGBoost', 'rmse': 0.58, 'r2': 0.89},
    ]
    records_df = pd.DataFrame(records)
    print('From list of dicts:')
    print(records_df)
    print()

    # Method 4: From a list of lists
    list_df = pd.DataFrame(
        [[1, 'A', 10.0], [2, 'B', 20.0], [3, 'C', 30.0]],
        columns=['id', 'category', 'value'],
    )
    print('From list of lists:')
    print(list_df)


demonstrate_dataframe_creation()

### 1.2 Indexing & Selection: loc, iloc, and Boolean IndexingPandas provides three main ways to select data:- **`df['col']`** or **`df.col`** — select a single column (returns Series)- **`df.loc[rows, cols]`** — label-based selection (by index/column names)- **`df.iloc[rows, cols]`** — position-based selection (by integer indices)- **`df[condition]`** — boolean indexing (filter rows)Understanding when to use each is critical for clean, bug-free data pipelines.

In [ ]:
def demonstrate_indexing(data: pd.DataFrame) -> None:
    """Show all major indexing patterns on the housing data.

    Args:
        data: DataFrame to demonstrate indexing on.
    """
    print('=== Column Selection ===')
    # Single column → Series
    income = data['MedInc']
    print(f'data["MedInc"]: type={type(income).__name__}, shape={income.shape}')
    print()

    # Multiple columns → DataFrame
    subset = data[['MedInc', 'HouseAge', 'MedHouseVal']]
    print(f'data[["MedInc", "HouseAge", "MedHouseVal"]]: shape={subset.shape}')
    print()

    print('=== loc (label-based) ===')
    # First 5 rows, specific columns
    loc_result = data.loc[:4, ['MedInc', 'HouseAge']]
    print(f'loc[:4, ["MedInc", "HouseAge"]]: shape={loc_result.shape}')
    print(loc_result)
    print()

    print('=== iloc (position-based) ===')
    # First 3 rows, first 2 columns
    iloc_result = data.iloc[:3, :2]
    print(f'iloc[:3, :2]: shape={iloc_result.shape}')
    print(iloc_result)
    print()

    print('=== Boolean Indexing (filtering) ===')
    # High income areas
    high_income = data[data['MedInc'] > 8.0]
    print(f'MedInc > 8.0: {len(high_income)} rows ({len(high_income)/len(data):.1%} of data)')

    # Combined conditions
    expensive_old = data[(data['MedHouseVal'] > 4.0) & (data['HouseAge'] > 40)]
    print(f'MedHouseVal > 4.0 AND HouseAge > 40: {len(expensive_old)} rows')

    # Using .query() for cleaner syntax
    query_result = data.query('MedInc > 5 and HouseAge < 20')
    print(f'query("MedInc > 5 and HouseAge < 20"): {len(query_result)} rows')
    print()

    print('=== Common Selection Patterns ===')
    patterns = pd.DataFrame({
        'Pattern': [
            'df["col"]', 'df[["a","b"]]', 'df.loc[row, col]',
            'df.iloc[i, j]', 'df[mask]', 'df.query("expr")',
        ],
        'Returns': [
            'Series', 'DataFrame', 'Scalar/Series/DF',
            'Scalar/Series/DF', 'DataFrame', 'DataFrame',
        ],
        'Use When': [
            'Need one column', 'Need multiple columns', 'Know label names',
            'Know positions', 'Filtering rows', 'Complex conditions',
        ],
    })
    print(patterns.to_string(index=False))


demonstrate_indexing(df)

### 1.3 Data Types and ConversionsPandas columns have specific data types (dtypes). Getting them right is crucial:wrong types waste memory, cause silent errors, and prevent proper operations.Common dtype issues:- Numbers stored as strings (after CSV loading)- Categorical data stored as generic `object` type- Dates stored as strings instead of `datetime64`- Using float64 when float32 suffices (2× memory waste)

In [ ]:
def demonstrate_dtypes() -> None:
    """Show dtype inspection, conversion, and memory impact."""
    # Create a messy DataFrame (simulating real-world data)
    messy = pd.DataFrame({
        'price': ['100.5', '200.0', '150.3', 'N/A', '300.0'],
        'category': ['A', 'B', 'A', 'C', 'B'],
        'date': ['2023-01-15', '2023-02-20', '2023-03-10', '2023-04-05', '2023-05-12'],
        'quantity': [10, 20, 15, 5, 25],
    })
    print('=== Before Type Conversion ===')
    print(messy.dtypes)
    print(f'Memory: {messy.memory_usage(deep=True).sum() / 1024:.2f} KB')
    print()

    # Fix types
    messy_fixed = messy.copy()
    messy_fixed['price'] = pd.to_numeric(messy_fixed['price'], errors='coerce')
    messy_fixed['category'] = messy_fixed['category'].astype('category')
    messy_fixed['date'] = pd.to_datetime(messy_fixed['date'])
    messy_fixed['quantity'] = messy_fixed['quantity'].astype(np.int16)

    print('=== After Type Conversion ===')
    print(messy_fixed.dtypes)
    print(f'Memory: {messy_fixed.memory_usage(deep=True).sum() / 1024:.2f} KB')
    print()
    print(messy_fixed)
    print()

    # Memory optimization on the housing dataset
    housing_f64 = df.copy()
    housing_f32 = df.astype(np.float32)
    mem_64 = housing_f64.memory_usage(deep=True).sum() / 1024 / 1024
    mem_32 = housing_f32.memory_usage(deep=True).sum() / 1024 / 1024
    print(f'Housing dataset memory:')
    print(f'  float64: {mem_64:.2f} MB')
    print(f'  float32: {mem_32:.2f} MB')
    print(f'  Savings: {(1 - mem_32/mem_64)*100:.0f}%')


demonstrate_dtypes()

### 1.4 Missing Values: Detection, Analysis, and TreatmentReal-world datasets almost always have missing values. The California Housingdataset is clean, so we'll artificially introduce missing values to demonstratethe full workflow.**Three strategies for handling missing data:**1. **Deletion** — drop rows or columns with too many missing values2. **Imputation** — fill with mean, median, mode, or more sophisticated methods3. **Indicator variables** — add a binary column marking where data was missingThe right strategy depends on *why* the data is missing (random vs systematic)and how much is missing.

In [ ]:
def create_dataset_with_missing(data: pd.DataFrame, frac: float = 0.1) -> pd.DataFrame:
    """Introduce random missing values into a DataFrame.

    Args:
        data: Original clean DataFrame.
        frac: Fraction of values to make missing.

    Returns:
        DataFrame with NaN values inserted randomly.
    """
    df_dirty = data.copy()
    np.random.seed(SEED)
    mask = np.random.random(df_dirty.shape) < frac
    df_dirty = df_dirty.mask(mask)
    return df_dirty


df_dirty = create_dataset_with_missing(df, frac=0.1)
print(f'Original shape: {df.shape}')
print(f'Missing values introduced: {df_dirty.isnull().sum().sum()} '
      f'({df_dirty.isnull().sum().sum() / df_dirty.size:.1%} of all values)')
print()
print('Missing values per column:')
print(df_dirty.isnull().sum().to_frame('missing').T)

In [ ]:
def analyze_missing_values(data: pd.DataFrame) -> pd.DataFrame:
    """Create a comprehensive missing value report.

    Args:
        data: DataFrame to analyze.

    Returns:
        DataFrame with missing value statistics per column.
    """
    total = len(data)
    missing = data.isnull().sum()
    pct = missing / total * 100

    report = pd.DataFrame({
        'Column': data.columns,
        'Missing': missing.values,
        'Pct Missing': pct.values,
        'dtype': data.dtypes.values,
        'Non-Null Mean': [data[c].mean() if data[c].dtype != object else None
                          for c in data.columns],
    })
    return report.sort_values('Pct Missing', ascending=False).reset_index(drop=True)


missing_report = analyze_missing_values(df_dirty)
print('=== Missing Value Report ===')
print(missing_report.to_string(index=False))

Now let's visualize the pattern of missing values. Understanding *where* datais missing helps decide whether deletion or imputation is appropriate.

In [ ]:
def visualize_missing(data: pd.DataFrame) -> None:
    """Visualize missing value patterns.

    Args:
        data: DataFrame to visualize.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Left: Missing values bar chart
    missing_pct = data.isnull().mean() * 100
    colors = ['#E53935' if p > 15 else '#FF9800' if p > 5 else '#43A047'
              for p in missing_pct]
    axes[0].barh(missing_pct.index, missing_pct.values, color=colors)
    axes[0].set_xlabel('% Missing')
    axes[0].set_title('Missing Values by Column')
    axes[0].axvline(x=5, color='gray', linestyle='--', alpha=0.5, label='5% threshold')
    axes[0].axvline(x=15, color='gray', linestyle=':', alpha=0.5, label='15% threshold')
    axes[0].legend()
    axes[0].grid(True, axis='x', alpha=0.3)

    # Right: Missing value heatmap (sample)
    sample = data.iloc[:100].isnull().astype(int)
    axes[1].imshow(sample.T, aspect='auto', cmap='Reds', interpolation='none')
    axes[1].set_yticks(range(len(data.columns)))
    axes[1].set_yticklabels(data.columns)
    axes[1].set_xlabel('Row Index (first 100)')
    axes[1].set_title('Missing Pattern (red = missing)')

    plt.tight_layout()
    plt.show()


visualize_missing(df_dirty)

With the missing value pattern understood, let's implement the three handlingstrategies and compare their effects on the data.

In [ ]:
def handle_missing_values(data: pd.DataFrame) -> dict[str, pd.DataFrame]:
    """Apply different missing value strategies and compare.

    Args:
        data: DataFrame with missing values.

    Returns:
        Dictionary mapping strategy name to cleaned DataFrame.
    """
    results: dict[str, pd.DataFrame] = {}

    # Strategy 1: Drop rows with any missing value
    drop_any = data.dropna()
    results['drop_any'] = drop_any
    print(f'Strategy 1 — Drop rows (any NaN):')
    print(f'  Rows: {len(data)} → {len(drop_any)} ({len(data)-len(drop_any)} dropped, '
          f'{(len(data)-len(drop_any))/len(data):.1%} loss)')
    print()

    # Strategy 2: Drop rows with threshold (keep if ≥70% non-null)
    thresh = int(data.shape[1] * 0.7)
    drop_thresh = data.dropna(thresh=thresh)
    results['drop_thresh'] = drop_thresh
    print(f'Strategy 2 — Drop rows (≥30% missing):')
    print(f'  Rows: {len(data)} → {len(drop_thresh)} ({len(data)-len(drop_thresh)} dropped)')
    print()

    # Strategy 3: Mean imputation
    mean_imputed = data.copy()
    for col in mean_imputed.select_dtypes(include=[np.number]).columns:
        mean_imputed[col] = mean_imputed[col].fillna(mean_imputed[col].mean())
    results['mean_impute'] = mean_imputed
    print(f'Strategy 3 — Mean imputation:')
    print(f'  Rows: {len(mean_imputed)} (all preserved)')
    print(f'  Remaining NaN: {mean_imputed.isnull().sum().sum()}')
    print()

    # Strategy 4: Median imputation
    median_imputed = data.copy()
    for col in median_imputed.select_dtypes(include=[np.number]).columns:
        median_imputed[col] = median_imputed[col].fillna(median_imputed[col].median())
    results['median_impute'] = median_imputed
    print(f'Strategy 4 — Median imputation:')
    print(f'  Rows: {len(median_imputed)} (all preserved)')
    print(f'  Remaining NaN: {median_imputed.isnull().sum().sum()}')
    print()

    # Strategy 5: Forward fill (useful for time series)
    ffill = data.ffill().bfill()  # Forward fill then backward fill edges
    results['ffill'] = ffill
    print(f'Strategy 5 — Forward/backward fill:')
    print(f'  Rows: {len(ffill)} (all preserved)')
    print(f'  Remaining NaN: {ffill.isnull().sum().sum()}')
    print()

    # Strategy 6: Imputation with missing indicator
    indicator = data.copy()
    for col in indicator.select_dtypes(include=[np.number]).columns:
        indicator[f'{col}_missing'] = indicator[col].isnull().astype(int)
        indicator[col] = indicator[col].fillna(indicator[col].median())
    results['indicator'] = indicator
    print(f'Strategy 6 — Median + missing indicator columns:')
    print(f'  Shape: {data.shape} → {indicator.shape} (added {indicator.shape[1] - data.shape[1]} indicator cols)')
    print(f'  Remaining NaN: {indicator.isnull().sum().sum()}')

    return results


strategies = handle_missing_values(df_dirty)

Let's compare how each strategy affects the distribution of a feature. Meanimputation changes the variance (concentrates values at the mean), whiledeletion can introduce selection bias.

In [ ]:
def compare_imputation_effects(
    original: pd.DataFrame,
    strategies: dict[str, pd.DataFrame],
    column: str,
) -> None:
    """Visualize how different imputation strategies affect a column's distribution.

    Args:
        original: Clean original DataFrame.
        strategies: Dictionary of strategy name to cleaned DataFrame.
        column: Column name to compare.
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 9))
    axes = axes.ravel()

    # Plot original
    axes[0].hist(original[column], bins=50, color='#1E88E5', alpha=0.7, density=True)
    axes[0].set_title(f'Original (n={len(original)})')
    axes[0].set_xlabel(column)
    axes[0].axvline(original[column].mean(), color='red', linestyle='--', label='mean')
    axes[0].legend()

    strategy_names = ['drop_any', 'drop_thresh', 'mean_impute',
                      'median_impute', 'ffill']
    colors = ['#E53935', '#FF9800', '#43A047', '#9C27B0', '#795548']

    for idx, (name, color) in enumerate(zip(strategy_names, colors)):
        ax = axes[idx + 1]
        strat_df = strategies[name]
        ax.hist(strat_df[column].dropna(), bins=50, color=color, alpha=0.7, density=True)
        ax.set_title(f'{name} (n={len(strat_df)})')
        ax.set_xlabel(column)
        ax.axvline(strat_df[column].mean(), color='red', linestyle='--', label='mean')
        ax.legend()

    plt.suptitle(f'Imputation Strategy Comparison: {column}', fontsize=13)
    plt.tight_layout()
    plt.show()

    # Quantitative comparison
    comp_records: list[dict] = []
    for name in ['original'] + strategy_names:
        data_col = original[column] if name == 'original' else strategies[name][column]
        comp_records.append({
            'Strategy': name,
            'Mean': data_col.mean(),
            'Std': data_col.std(),
            'Median': data_col.median(),
            'Count': len(data_col.dropna()),
        })
    comp_df = pd.DataFrame(comp_records)
    print(f'=== Distribution Comparison: {column} ===')
    print(comp_df.to_string(index=False))


compare_imputation_effects(df, strategies, 'MedInc')

### 1.5 Categorical Encoding: Converting Categories to NumbersML models need numerical input. When data has categorical features (likecity names, product types, or education levels), we must encode them as numbers.**Encoding strategies:**- **Label encoding** — assign integer 0, 1, 2, ... (for ordinal data)- **One-hot encoding** — binary column per category (for nominal data)- **Ordinal encoding** — map to integers respecting order (low/medium/high)- **Target encoding** — replace category with mean target value (powerful but leaky)- **Frequency encoding** — replace category with its frequencyThe California Housing dataset is purely numerical, so we'll create a syntheticcategorical version to demonstrate encoding.

In [ ]:
def create_categorical_dataset(data: pd.DataFrame) -> pd.DataFrame:
    """Add synthetic categorical features to demonstrate encoding.

    Args:
        data: Original numerical DataFrame.

    Returns:
        DataFrame with additional categorical columns.
    """
    df_cat = data.copy()

    # Create a categorical 'income_bracket' from MedInc
    df_cat['income_bracket'] = pd.cut(
        df_cat['MedInc'],
        bins=[0, 2, 4, 6, 8, np.inf],
        labels=['very_low', 'low', 'medium', 'high', 'very_high'],
    )

    # Create 'house_age_category'
    df_cat['age_category'] = pd.cut(
        df_cat['HouseAge'],
        bins=[0, 15, 30, 45, np.inf],
        labels=['new', 'moderate', 'old', 'very_old'],
    )

    # Create a 'region' based on latitude/longitude
    np.random.seed(SEED)
    regions = ['Bay Area', 'LA Metro', 'San Diego', 'Sacramento', 'Central Valley']
    df_cat['region'] = np.random.choice(regions, size=len(df_cat))

    print(f'Added categorical columns:')
    for col in ['income_bracket', 'age_category', 'region']:
        print(f'  {col}: {df_cat[col].nunique()} categories — {df_cat[col].value_counts().head(3).to_dict()}')

    return df_cat


df_cat = create_categorical_dataset(df)
print()
df_cat[['MedInc', 'income_bracket', 'HouseAge', 'age_category', 'region', 'MedHouseVal']].head(10)

In [ ]:
def demonstrate_encoding_methods(data: pd.DataFrame) -> dict[str, pd.DataFrame]:
    """Apply and compare different categorical encoding methods.

    Args:
        data: DataFrame with categorical columns.

    Returns:
        Dictionary mapping encoding name to encoded DataFrame.
    """
    results: dict[str, pd.DataFrame] = {}
    cat_cols = ['income_bracket', 'age_category', 'region']

    # 1. Label Encoding
    label_encoded = data.copy()
    label_maps: dict[str, dict] = {}
    for col in cat_cols:
        categories = label_encoded[col].astype(str).unique()
        mapping = {cat: idx for idx, cat in enumerate(sorted(categories))}
        label_maps[col] = mapping
        label_encoded[f'{col}_label'] = label_encoded[col].astype(str).map(mapping)
    results['label'] = label_encoded
    print('1. Label Encoding:')
    for col, mapping in label_maps.items():
        print(f'   {col}: {mapping}')
    print()

    # 2. One-Hot Encoding
    onehot_encoded = pd.get_dummies(data, columns=cat_cols, prefix=cat_cols, dtype=int)
    results['onehot'] = onehot_encoded
    new_cols = [c for c in onehot_encoded.columns if c not in data.columns]
    print(f'2. One-Hot Encoding:')
    print(f'   Original columns: {len(data.columns)}')
    print(f'   After one-hot: {len(onehot_encoded.columns)} (+{len(new_cols)} dummy columns)')
    print(f'   New columns: {new_cols[:6]}...')
    print()

    # 3. Ordinal Encoding (for ordered categories)
    ordinal_encoded = data.copy()
    ordinal_maps = {
        'income_bracket': {'very_low': 0, 'low': 1, 'medium': 2, 'high': 3, 'very_high': 4},
        'age_category': {'new': 0, 'moderate': 1, 'old': 2, 'very_old': 3},
    }
    for col, mapping in ordinal_maps.items():
        ordinal_encoded[f'{col}_ordinal'] = ordinal_encoded[col].astype(str).map(mapping)
    results['ordinal'] = ordinal_encoded
    print(f'3. Ordinal Encoding (preserves order):')
    for col, mapping in ordinal_maps.items():
        print(f'   {col}: {mapping}')
    print()

    # 4. Target Encoding (mean of target per category)
    target_encoded = data.copy()
    target_col = 'MedHouseVal'
    print(f'4. Target Encoding (mean {target_col} per category):')
    for col in cat_cols:
        means = data.groupby(col)[target_col].mean()
        target_encoded[f'{col}_target'] = target_encoded[col].astype(str).map(means)
        print(f'   {col}: {means.to_dict()}')
    results['target'] = target_encoded
    print()

    # 5. Frequency Encoding
    freq_encoded = data.copy()
    print(f'5. Frequency Encoding:')
    for col in cat_cols:
        freq = data[col].value_counts(normalize=True)
        freq_encoded[f'{col}_freq'] = freq_encoded[col].astype(str).map(freq)
        print(f'   {col}: {freq.head(3).to_dict()}')
    results['frequency'] = freq_encoded

    return results


encodings = demonstrate_encoding_methods(df_cat)

Let's compare the shape and memory impact of each encoding method. One-hotencoding creates the most columns, which can be problematic with high-cardinalityfeatures.

In [ ]:
def compare_encodings(encodings: dict[str, pd.DataFrame]) -> pd.DataFrame:
    """Compare encoding methods by shape and memory.

    Args:
        encodings: Dictionary of encoding name to encoded DataFrame.

    Returns:
        Comparison DataFrame.
    """
    records: list[dict] = []
    for name, encoded_df in encodings.items():
        records.append({
            'Encoding': name,
            'Columns': len(encoded_df.columns),
            'Memory (KB)': encoded_df.memory_usage(deep=True).sum() / 1024,
            'New Columns': len(encoded_df.columns) - len(df_cat.columns),
        })
    return pd.DataFrame(records)


enc_comparison = compare_encodings(encodings)
print('=== Encoding Method Comparison ===')
print(enc_comparison.to_string(index=False))
print()
print('Trade-offs:')
print('  Label: Compact but implies false ordering for nominal data')
print('  One-hot: No false ordering but high dimensionality')
print('  Ordinal: Perfect for truly ordered categories')
print('  Target: Powerful but risk of target leakage (use with CV folds)')
print('  Frequency: Captures rarity without leaking target info')

### 1.6 Feature Engineering: Creating Informative FeaturesFeature engineering transforms raw columns into features that better capturethe patterns in the data. Common techniques include:- **Mathematical transforms** — log, square root, reciprocal- **Interaction features** — products or ratios of existing features- **Binning** — converting continuous to categorical- **Aggregation** — group-level statistics (mean, count per category)

In [ ]:
def engineer_features(data: pd.DataFrame) -> pd.DataFrame:
    """Create engineered features from the housing dataset.

    Args:
        data: Original housing DataFrame.

    Returns:
        DataFrame with additional engineered features.
    """
    df_eng = data.copy()

    # Log transforms (reduce right skew)
    df_eng['log_MedInc'] = np.log1p(df_eng['MedInc'])
    df_eng['log_Population'] = np.log1p(df_eng['Population'])

    # Interaction features
    df_eng['rooms_per_household'] = df_eng['AveRooms'] / df_eng['AveOccup'].clip(lower=0.1)
    df_eng['bedrooms_ratio'] = df_eng['AveBedrms'] / df_eng['AveRooms'].clip(lower=0.1)
    df_eng['income_per_room'] = df_eng['MedInc'] / df_eng['AveRooms'].clip(lower=0.1)

    # Binning
    df_eng['income_bin'] = pd.qcut(df_eng['MedInc'], q=5, labels=[1, 2, 3, 4, 5])

    # Polynomial features
    df_eng['MedInc_squared'] = df_eng['MedInc'] ** 2

    # Distance from major city (rough approximation using lat/lon)
    la_lat, la_lon = 34.05, -118.24
    sf_lat, sf_lon = 37.77, -122.42
    df_eng['dist_to_LA'] = np.sqrt(
        (df_eng['Latitude'] - la_lat)**2 + (df_eng['Longitude'] - la_lon)**2)
    df_eng['dist_to_SF'] = np.sqrt(
        (df_eng['Latitude'] - sf_lat)**2 + (df_eng['Longitude'] - sf_lon)**2)
    df_eng['min_city_dist'] = df_eng[['dist_to_LA', 'dist_to_SF']].min(axis=1)

    print(f'Original features: {len(data.columns)}')
    new_features = [c for c in df_eng.columns if c not in data.columns]
    print(f'Engineered features: {len(new_features)}')
    print(f'Total features: {len(df_eng.columns)}')
    print()
    for feat in new_features:
        vals = df_eng[feat].dropna()
        print(f'  {feat:25s}: mean={vals.mean():8.3f}, std={vals.std():8.3f}')

    return df_eng


df_engineered = engineer_features(df)

Let's visualize how feature engineering improves the relationship with thetarget variable. The log transform, for instance, often linearizes skewedrelationships.

In [ ]:
def visualize_feature_engineering(original: pd.DataFrame, engineered: pd.DataFrame) -> None:
    """Compare original vs engineered features' relationship with target.

    Args:
        original: Original DataFrame.
        engineered: DataFrame with engineered features.
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 9))

    target = 'MedHouseVal'
    sample_idx = np.random.choice(len(original), 2000, replace=False)

    # Row 1: Original features
    pairs_orig = [('MedInc', target), ('Population', target), ('AveRooms', target)]
    for idx, (feat, tgt) in enumerate(pairs_orig):
        axes[0, idx].scatter(original[feat].iloc[sample_idx],
                            original[tgt].iloc[sample_idx],
                            s=5, alpha=0.3, color='#1E88E5')
        axes[0, idx].set_xlabel(feat)
        axes[0, idx].set_ylabel(tgt)
        axes[0, idx].set_title(f'Original: {feat}')

    # Row 2: Engineered features
    pairs_eng = [('log_MedInc', target), ('log_Population', target),
                 ('rooms_per_household', target)]
    for idx, (feat, tgt) in enumerate(pairs_eng):
        vals = engineered[feat].iloc[sample_idx]
        # Clip extreme values for better visualization
        lower, upper = vals.quantile(0.01), vals.quantile(0.99)
        mask = (vals >= lower) & (vals <= upper)
        axes[1, idx].scatter(vals[mask],
                            engineered[tgt].iloc[sample_idx][mask],
                            s=5, alpha=0.3, color='#43A047')
        axes[1, idx].set_xlabel(feat)
        axes[1, idx].set_ylabel(tgt)
        axes[1, idx].set_title(f'Engineered: {feat}')

    plt.suptitle('Feature Engineering: Original vs Transformed', fontsize=13)
    plt.tight_layout()
    plt.show()


visualize_feature_engineering(df, df_engineered)

---## Part 2 — Putting It All Together: DataPipeline ClassWe've built individual preprocessing components. Now let's assemble them intoa reusable `DataPipeline` class that handles the full EDA → clean → encode →engineer workflow in a structured, reproducible way.

In [ ]:
class DataPipeline:
    """End-to-end data preprocessing pipeline for tabular ML.

    Handles missing values, categorical encoding, feature engineering,
    and train/test splitting in a reproducible manner.

    Attributes:
        impute_strategy: Strategy for handling missing values.
        encoding_method: Strategy for encoding categorical features.
        numeric_columns: List of numeric column names after fitting.
        categorical_columns: List of categorical column names after fitting.
        impute_values: Dictionary of column → imputation value (after fit).
        encoding_maps: Dictionary of column → encoding mapping (after fit).
    """

    def __init__(
        self,
        impute_strategy: str = 'median',
        encoding_method: str = 'onehot',
    ) -> None:
        """Initialize pipeline with strategy choices.

        Args:
            impute_strategy: 'mean', 'median', or 'drop'.
            encoding_method: 'onehot', 'label', or 'ordinal'.
        """
        self.impute_strategy = impute_strategy
        self.encoding_method = encoding_method
        self.numeric_columns: list[str] = []
        self.categorical_columns: list[str] = []
        self.impute_values: dict[str, float] = {}
        self.encoding_maps: dict[str, dict] = {}
        self._fitted = False

    def fit(self, data: pd.DataFrame, target_col: str | None = None) -> 'DataPipeline':
        """Learn preprocessing parameters from training data.

        Args:
            data: Training DataFrame.
            target_col: Name of target column (excluded from features).

        Returns:
            Self for method chaining.
        """
        feature_cols = [c for c in data.columns if c != target_col]

        # Identify column types
        self.numeric_columns = [
            c for c in feature_cols
            if data[c].dtype in [np.float64, np.float32, np.int64, np.int32]
        ]
        self.categorical_columns = [
            c for c in feature_cols
            if data[c].dtype in ['object', 'category']
        ]

        # Learn imputation values
        for col in self.numeric_columns:
            if self.impute_strategy == 'mean':
                self.impute_values[col] = data[col].mean()
            elif self.impute_strategy == 'median':
                self.impute_values[col] = data[col].median()

        # Learn encoding maps
        for col in self.categorical_columns:
            categories = sorted(data[col].astype(str).unique())
            self.encoding_maps[col] = {cat: idx for idx, cat in enumerate(categories)}

        self._fitted = True
        return self

    def transform(self, data: pd.DataFrame) -> pd.DataFrame:
        """Apply learned preprocessing to data.

        Args:
            data: DataFrame to transform.

        Returns:
            Preprocessed DataFrame with only numeric columns.
        """
        assert self._fitted, 'Pipeline must be fit before transform'
        result = data.copy()

        # Impute missing values
        if self.impute_strategy == 'drop':
            result = result.dropna(subset=self.numeric_columns)
        else:
            for col in self.numeric_columns:
                if col in result.columns:
                    result[col] = result[col].fillna(self.impute_values.get(col, 0))

        # Encode categoricals
        if self.encoding_method == 'onehot':
            result = pd.get_dummies(
                result, columns=self.categorical_columns,
                prefix=self.categorical_columns, dtype=int,
            )
        elif self.encoding_method == 'label':
            for col in self.categorical_columns:
                if col in result.columns:
                    mapping = self.encoding_maps[col]
                    result[col] = result[col].astype(str).map(mapping).fillna(-1).astype(int)

        return result

    def fit_transform(self, data: pd.DataFrame, target_col: str | None = None) -> pd.DataFrame:
        """Fit and transform in one step.

        Args:
            data: Training DataFrame.
            target_col: Name of target column.

        Returns:
            Preprocessed DataFrame.
        """
        return self.fit(data, target_col).transform(data)

    def summary(self) -> None:
        """Print pipeline configuration summary."""
        assert self._fitted, 'Pipeline not fitted yet'
        print(f'=== DataPipeline Summary ===')
        print(f'  Imputation: {self.impute_strategy}')
        print(f'  Encoding: {self.encoding_method}')
        print(f'  Numeric columns: {len(self.numeric_columns)}')
        print(f'  Categorical columns: {len(self.categorical_columns)}')
        if self.impute_values:
            print(f'  Impute values: {dict(list(self.impute_values.items())[:3])}...')

Let's test the pipeline on our data with different configurations.

In [ ]:
def test_pipeline() -> None:
    """Test DataPipeline with various configurations."""
    # Create test data with categoricals and missing values
    test_data = create_dataset_with_missing(df_cat, frac=0.05)

    # Pipeline 1: Median imputation + one-hot encoding
    pipe1 = DataPipeline(impute_strategy='median', encoding_method='onehot')
    result1 = pipe1.fit_transform(test_data, target_col='MedHouseVal')
    pipe1.summary()
    print(f'  Output shape: {result1.shape}')
    print(f'  Missing values: {result1.isnull().sum().sum()}')
    print()

    # Pipeline 2: Mean imputation + label encoding
    pipe2 = DataPipeline(impute_strategy='mean', encoding_method='label')
    result2 = pipe2.fit_transform(test_data, target_col='MedHouseVal')
    pipe2.summary()
    print(f'  Output shape: {result2.shape}')
    print(f'  Missing values: {result2.isnull().sum().sum()}')
    print()

    # Pipeline 3: Drop missing
    pipe3 = DataPipeline(impute_strategy='drop', encoding_method='onehot')
    result3 = pipe3.fit_transform(test_data, target_col='MedHouseVal')
    pipe3.summary()
    print(f'  Output shape: {result3.shape}')
    print(f'  Rows dropped: {len(test_data) - len(result3)}')


test_pipeline()

---## Part 3 — Application: Complete EDA & Preprocessing PipelineNow we apply everything to the California Housing dataset in a realisticworkflow: full EDA, correlation analysis, preprocessing, and conversionto NumPy/PyTorch for model training.

### 3.1 Comprehensive EDAA thorough EDA explores distributions, correlations, and outliers beforeany modeling. This prevents surprises later.

In [ ]:
def comprehensive_eda(data: pd.DataFrame, target: str) -> None:
    """Perform comprehensive exploratory data analysis.

    Args:
        data: DataFrame to analyze.
        target: Name of the target column.
    """
    print('=== Dataset Overview ===')
    print(f'Shape: {data.shape}')
    print(f'Memory: {data.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB')
    print(f'Duplicates: {data.duplicated().sum()}')
    print()

    # Distribution summary
    print('=== Feature Distributions ===')
    desc = data.describe().T
    desc['skew'] = data.skew()
    desc['kurtosis'] = data.kurtosis()
    desc['iqr'] = desc['75%'] - desc['25%']
    print(desc[['mean', 'std', 'min', 'max', 'skew', 'kurtosis']].to_string())
    print()

    # Outlier detection using IQR
    print('=== Outliers (IQR Method) ===')
    for col in data.select_dtypes(include=[np.number]).columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((data[col] < Q1 - 1.5 * IQR) | (data[col] > Q3 + 1.5 * IQR)).sum()
        if outliers > 0:
            print(f'  {col}: {outliers} outliers ({outliers/len(data):.1%})')


comprehensive_eda(df, 'MedHouseVal')

In [ ]:
def plot_distributions(data: pd.DataFrame) -> None:
    """Plot distribution of all numeric features.

    Args:
        data: DataFrame to visualize.
    """
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    n_cols = 3
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes = axes.ravel()

    for idx, col in enumerate(numeric_cols):
        axes[idx].hist(data[col], bins=50, color='#1E88E5', alpha=0.7, edgecolor='white')
        axes[idx].set_title(f'{col} (skew={data[col].skew():.2f})')
        axes[idx].set_xlabel(col)
        axes[idx].axvline(data[col].mean(), color='red', linestyle='--',
                          label=f'mean={data[col].mean():.2f}')
        axes[idx].axvline(data[col].median(), color='green', linestyle='--',
                          label=f'median={data[col].median():.2f}')
        axes[idx].legend(fontsize=8)

    # Hide empty subplots
    for idx in range(len(numeric_cols), len(axes)):
        axes[idx].set_visible(False)

    plt.suptitle('Feature Distributions', fontsize=14)
    plt.tight_layout()
    plt.show()


plot_distributions(df)

### 3.2 Correlation AnalysisUnderstanding correlations between features and with the target helps us:- Identify the most predictive features- Detect multicollinearity (redundant features)- Guide feature selection

In [ ]:
def correlation_analysis(data: pd.DataFrame, target: str) -> None:
    """Analyze and visualize feature correlations.

    Args:
        data: DataFrame to analyze.
        target: Target column name.
    """
    corr_matrix = data.select_dtypes(include=[np.number]).corr()

    # Correlation with target
    target_corr = corr_matrix[target].drop(target).sort_values(ascending=False)
    print(f'=== Correlation with {target} ===')
    for feat, corr_val in target_corr.items():
        bar = '█' * int(abs(corr_val) * 20)
        sign = '+' if corr_val > 0 else '-'
        print(f'  {feat:20s}: {corr_val:+.4f} {sign}{bar}')
    print()

    # Correlation heatmap
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Full heatmap
    im = axes[0].imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1, aspect='auto')
    axes[0].set_xticks(range(len(corr_matrix)))
    axes[0].set_xticklabels(corr_matrix.columns, rotation=45, ha='right', fontsize=8)
    axes[0].set_yticks(range(len(corr_matrix)))
    axes[0].set_yticklabels(corr_matrix.columns, fontsize=8)
    axes[0].set_title('Correlation Heatmap')
    plt.colorbar(im, ax=axes[0], shrink=0.8)

    # Annotate significant correlations
    for i in range(len(corr_matrix)):
        for j in range(len(corr_matrix)):
            val = corr_matrix.iloc[i, j]
            if abs(val) > 0.3 and i != j:
                axes[0].text(j, i, f'{val:.2f}', ha='center', va='center', fontsize=7)

    # Bar chart of target correlations
    colors = ['#43A047' if v > 0 else '#E53935' for v in target_corr.values]
    axes[1].barh(target_corr.index, target_corr.values, color=colors)
    axes[1].set_xlabel(f'Correlation with {target}')
    axes[1].set_title(f'Feature Importance (by correlation)')
    axes[1].axvline(x=0, color='gray', linestyle='-', alpha=0.3)
    axes[1].grid(True, axis='x', alpha=0.3)

    plt.tight_layout()
    plt.show()


correlation_analysis(df, 'MedHouseVal')

### 3.3 GroupBy and AggregationPandas' `groupby` is one of its most powerful features. It allows split-apply-combineoperations that are essential for understanding data subgroups and creatingaggregate features.

In [ ]:
def demonstrate_groupby(data: pd.DataFrame) -> None:
    """Show groupby patterns commonly used in ML feature engineering.

    Args:
        data: DataFrame with categorical and numeric columns.
    """
    # Add income bracket for grouping
    df_grouped = data.copy()
    df_grouped['income_bracket'] = pd.cut(
        df_grouped['MedInc'], bins=[0, 2, 4, 6, 8, np.inf],
        labels=['very_low', 'low', 'medium', 'high', 'very_high'])

    # Basic groupby: mean target by income bracket
    print('=== Mean House Value by Income Bracket ===')
    group_means = df_grouped.groupby('income_bracket', observed=True)['MedHouseVal'].agg(
        ['mean', 'median', 'std', 'count']
    )
    print(group_means.to_string())
    print()

    # Multiple aggregations
    print('=== Multi-Column Aggregation ===')
    agg_result = df_grouped.groupby('income_bracket', observed=True).agg({
        'MedHouseVal': ['mean', 'std'],
        'HouseAge': 'mean',
        'AveRooms': 'mean',
        'Population': 'sum',
    })
    # Flatten multi-level columns
    agg_result.columns = ['_'.join(col).strip() for col in agg_result.columns]
    print(agg_result.to_string())
    print()

    # Transform: add group statistics back to original rows
    df_grouped['bracket_mean_val'] = df_grouped.groupby(
        'income_bracket', observed=True
    )['MedHouseVal'].transform('mean')
    df_grouped['bracket_std_val'] = df_grouped.groupby(
        'income_bracket', observed=True
    )['MedHouseVal'].transform('std')
    df_grouped['val_zscore_in_bracket'] = (
        (df_grouped['MedHouseVal'] - df_grouped['bracket_mean_val'])
        / df_grouped['bracket_std_val']
    )
    print('=== Group-Level Features (transform) ===')
    print(df_grouped[['income_bracket', 'MedHouseVal', 'bracket_mean_val',
                       'val_zscore_in_bracket']].head(10).to_string())


demonstrate_groupby(df)

### 3.4 Pandas ↔ NumPy ↔ PyTorch ConversionThe final step in any data pipeline is converting the processed DataFrameinto the format your model expects. For classical ML, that's NumPy arrays.For deep learning, that's PyTorch tensors.

In [ ]:
def demonstrate_conversions(data: pd.DataFrame, target_col: str) -> None:
    """Show conversion between Pandas, NumPy, and PyTorch.

    Args:
        data: Preprocessed DataFrame.
        target_col: Name of the target column.
    """
    features = data.drop(columns=[target_col]).select_dtypes(include=[np.number])
    target = data[target_col]

    # ── Pandas → NumPy ──────────────────────────────────────────────────────
    X_np = features.values  # or features.to_numpy()
    y_np = target.values
    print('=== Pandas → NumPy ===')
    print(f'  X: {X_np.shape}, dtype={X_np.dtype}')
    print(f'  y: {y_np.shape}, dtype={y_np.dtype}')
    print()

    # ── NumPy → PyTorch ─────────────────────────────────────────────────────
    X_tensor = torch.from_numpy(X_np.astype(np.float32))
    y_tensor = torch.from_numpy(y_np.astype(np.float32))
    print('=== NumPy → PyTorch ===')
    print(f'  X: {X_tensor.shape}, dtype={X_tensor.dtype}')
    print(f'  y: {y_tensor.shape}, dtype={y_tensor.dtype}')
    print()

    # ── Train/Test Split (sklearn convention for Module 1-4) ────────────────
    X_train, X_test, y_train, y_test = train_test_split(
        X_np, y_np, test_size=0.2, random_state=SEED)
    print('=== Train/Test Split (80/20) ===')
    print(f'  X_train: {X_train.shape}')
    print(f'  X_test:  {X_test.shape}')
    print(f'  y_train: {y_train.shape}')
    print(f'  y_test:  {y_test.shape}')
    print()

    # ── PyTorch → Pandas (for analysis after model predictions) ─────────────
    predictions = torch.randn(len(X_test))  # Simulated predictions
    results_df = pd.DataFrame({
        'Actual': y_test,
        'Predicted': predictions.numpy(),
        'Error': y_test - predictions.numpy(),
    })
    print('=== PyTorch → Pandas (analysis) ===')
    print(results_df.describe().to_string())
    print()

    # Memory comparison
    mem_pandas = features.memory_usage(deep=True).sum()
    mem_numpy = X_np.nbytes
    mem_torch = X_tensor.element_size() * X_tensor.nelement()
    print('=== Memory Comparison ===')
    mem_df = pd.DataFrame({
        'Format': ['Pandas (float64)', 'NumPy (float64)', 'PyTorch (float32)'],
        'Memory (MB)': [mem_pandas / 1e6, mem_numpy / 1e6, mem_torch / 1e6],
    })
    print(mem_df.to_string(index=False))


demonstrate_conversions(df, 'MedHouseVal')

---## Part 4 — Evaluation & AnalysisLet's evaluate our preprocessing pipeline by measuring its impact on a simplemodel, analyzing common errors, and benchmarking Pandas operations.

### 4.1 Pipeline Impact on Model PerformanceTo see if our preprocessing actually helps, we'll train a simple linearregression on raw vs preprocessed data and compare results.

In [ ]:
def evaluate_pipeline_impact() -> pd.DataFrame:
    """Compare model performance with different preprocessing strategies.

    Returns:
        DataFrame with performance comparison.
    """
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error, r2_score
    from sklearn.preprocessing import StandardScaler

    target_col = 'MedHouseVal'
    records: list[dict] = []

    # Raw data (no preprocessing)
    X_raw = df.drop(columns=[target_col]).values
    y = df[target_col].values
    X_train, X_test, y_train, y_test = train_test_split(
        X_raw, y, test_size=0.2, random_state=SEED)

    model_raw = LinearRegression().fit(X_train, y_train)
    pred_raw = model_raw.predict(X_test)
    records.append({
        'Pipeline': 'Raw (no preprocessing)',
        'RMSE': np.sqrt(mean_squared_error(y_test, pred_raw)),
        'R²': r2_score(y_test, pred_raw),
        'Features': X_raw.shape[1],
    })

    # Standardized data
    scaler = StandardScaler()
    X_scaled_train = scaler.fit_transform(X_train)
    X_scaled_test = scaler.transform(X_test)

    model_scaled = LinearRegression().fit(X_scaled_train, y_train)
    pred_scaled = model_scaled.predict(X_scaled_test)
    records.append({
        'Pipeline': 'StandardScaler',
        'RMSE': np.sqrt(mean_squared_error(y_test, pred_scaled)),
        'R²': r2_score(y_test, pred_scaled),
        'Features': X_scaled_train.shape[1],
    })

    # With engineered features
    df_eng = engineer_features(df)
    numeric_eng = df_eng.select_dtypes(include=[np.number]).drop(columns=[target_col])
    # Remove any inf values
    numeric_eng = numeric_eng.replace([np.inf, -np.inf], np.nan).fillna(numeric_eng.median())
    X_eng = numeric_eng.values
    X_eng_train, X_eng_test, y_eng_train, y_eng_test = train_test_split(
        X_eng, y, test_size=0.2, random_state=SEED)

    scaler_eng = StandardScaler()
    X_eng_train_s = scaler_eng.fit_transform(X_eng_train)
    X_eng_test_s = scaler_eng.transform(X_eng_test)

    model_eng = LinearRegression().fit(X_eng_train_s, y_eng_train)
    pred_eng = model_eng.predict(X_eng_test_s)
    records.append({
        'Pipeline': 'Scaled + Engineered Features',
        'RMSE': np.sqrt(mean_squared_error(y_eng_test, pred_eng)),
        'R²': r2_score(y_eng_test, pred_eng),
        'Features': X_eng_train_s.shape[1],
    })

    return pd.DataFrame(records)


impact_df = evaluate_pipeline_impact()
print('=== Preprocessing Impact on Linear Regression ===')
print(impact_df.to_string(index=False))
print()
print('Feature engineering improves R² because the engineered features')
print('(log transforms, interaction terms, distance) capture non-linear')
print('relationships that raw linear regression cannot learn.')

### 4.2 Error Analysis: Where Does Preprocessing Fail?Let's examine which samples have the largest prediction errors and whetherthey share common characteristics — this reveals preprocessing gaps.

In [ ]:
def error_analysis() -> None:
    """Analyze prediction errors to identify preprocessing weaknesses."""
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import StandardScaler

    target_col = 'MedHouseVal'
    X = df.drop(columns=[target_col]).values
    y = df[target_col].values

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=SEED)

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    model = LinearRegression().fit(X_train_s, y_train)
    preds = model.predict(X_test_s)
    errors = y_test - preds
    abs_errors = np.abs(errors)

    # Create analysis DataFrame
    feature_names = df.drop(columns=[target_col]).columns
    analysis = pd.DataFrame(X_test, columns=feature_names)
    analysis['actual'] = y_test
    analysis['predicted'] = preds
    analysis['error'] = errors
    analysis['abs_error'] = abs_errors

    # Error distribution
    print('=== Error Distribution ===')
    print(f'  Mean Error: {errors.mean():.4f}')
    print(f'  Std Error: {errors.std():.4f}')
    print(f'  Mean Abs Error: {abs_errors.mean():.4f}')
    print(f'  Median Abs Error: {np.median(abs_errors):.4f}')
    print(f'  Max Abs Error: {abs_errors.max():.4f}')
    print()

    # Worst predictions
    worst = analysis.nlargest(10, 'abs_error')
    print('=== Top 10 Worst Predictions ===')
    print(worst[['MedInc', 'HouseAge', 'AveRooms', 'actual', 'predicted', 'error']].to_string())
    print()

    # Error by income bracket
    analysis['income_bracket'] = pd.cut(
        analysis['MedInc'], bins=[0, 2, 4, 6, 8, np.inf],
        labels=['very_low', 'low', 'medium', 'high', 'very_high'])
    print('=== Error by Income Bracket ===')
    error_by_bracket = analysis.groupby('income_bracket', observed=True)['abs_error'].agg(
        ['mean', 'median', 'count']).round(4)
    print(error_by_bracket.to_string())
    print()
    print('Higher income brackets tend to have larger errors because the')
    print('relationship between income and house value is non-linear.')

    # Visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    axes[0].hist(errors, bins=50, color='#1E88E5', alpha=0.7, edgecolor='white')
    axes[0].set_xlabel('Prediction Error')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Error Distribution')
    axes[0].axvline(0, color='red', linestyle='--')

    axes[1].scatter(y_test, preds, s=3, alpha=0.3, color='#1E88E5')
    axes[1].plot([0, 5], [0, 5], 'r--', linewidth=2, label='Perfect')
    axes[1].set_xlabel('Actual')
    axes[1].set_ylabel('Predicted')
    axes[1].set_title('Predicted vs Actual')
    axes[1].legend()

    axes[2].scatter(analysis['MedInc'], abs_errors, s=3, alpha=0.3, color='#E53935')
    axes[2].set_xlabel('Median Income')
    axes[2].set_ylabel('Absolute Error')
    axes[2].set_title('Error vs Income')

    plt.tight_layout()
    plt.show()


error_analysis()

### 4.3 Performance Benchmark: Pandas OperationsLet's benchmark common Pandas operations to understand their computational cost.This helps decide when to convert to NumPy for performance-critical code.

In [ ]:
def benchmark_pandas_ops() -> pd.DataFrame:
    """Benchmark common Pandas operations.

    Returns:
        DataFrame with timing results.
    """
    # Use a larger DataFrame for meaningful benchmarks
    np.random.seed(SEED)
    n = 100_000
    big_df = pd.DataFrame({
        'A': np.random.randn(n),
        'B': np.random.randn(n),
        'C': np.random.randn(n),
        'cat': np.random.choice(['x', 'y', 'z'], n),
    })
    big_np = big_df[['A', 'B', 'C']].values

    records: list[dict] = []

    def measure(name: str, func: callable) -> None:
        """Measure and record timing."""
        for _ in range(3):
            func()
        times = []
        for _ in range(5):
            start = time.perf_counter()
            func()
            times.append(time.perf_counter() - start)
        records.append({'Operation': name, 'Time (ms)': np.mean(times) * 1000})

    # Pandas operations
    measure('df.sum()', lambda: big_df[['A', 'B', 'C']].sum())
    measure('df.mean()', lambda: big_df[['A', 'B', 'C']].mean())
    measure('df.describe()', lambda: big_df.describe())
    measure('df.groupby().mean()', lambda: big_df.groupby('cat')[['A', 'B']].mean())
    measure('df.sort_values()', lambda: big_df.sort_values('A'))
    measure('df[mask] filter', lambda: big_df[big_df['A'] > 0])
    measure('df.apply(lambda)', lambda: big_df['A'].apply(lambda x: x ** 2))
    measure('df.fillna()', lambda: big_df.fillna(0))

    # NumPy equivalents
    measure('np.sum()', lambda: np.sum(big_np, axis=0))
    measure('np.mean()', lambda: np.mean(big_np, axis=0))
    measure('np.sort()', lambda: np.sort(big_np[:, 0]))
    measure('np[mask] filter', lambda: big_np[big_np[:, 0] > 0])
    measure('np ** 2', lambda: big_np[:, 0] ** 2)

    return pd.DataFrame(records)


ops_bench = benchmark_pandas_ops()
print('=== Pandas vs NumPy Operation Speed (n=100K) ===')
print(ops_bench.to_string(index=False))
print()
print('Pandas adds overhead for index management and type checking.')
print('For numerical-only operations, NumPy is faster.')
print('Use Pandas for data wrangling; convert to NumPy for computation.')

### 4.4 Merge and Join OperationsIn practice, ML features often come from multiple tables that must be joined.Pandas provides SQL-style merge operations for combining DataFrames.

In [ ]:
def demonstrate_merge_operations() -> None:
    """Show merge/join patterns common in ML feature pipelines."""
    # Create sample tables
    customers = pd.DataFrame({
        'customer_id': [1, 2, 3, 4, 5],
        'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
        'segment': ['premium', 'basic', 'premium', 'basic', 'premium'],
    })

    orders = pd.DataFrame({
        'order_id': [101, 102, 103, 104, 105, 106],
        'customer_id': [1, 2, 1, 3, 2, 6],  # customer 6 doesn't exist
        'amount': [100.0, 50.0, 200.0, 150.0, 75.0, 30.0],
    })

    print('=== customers ===')
    print(customers.to_string(index=False))
    print()
    print('=== orders ===')
    print(orders.to_string(index=False))
    print()

    # Inner join: only matching rows
    inner = pd.merge(customers, orders, on='customer_id', how='inner')
    print(f'Inner join: {len(inner)} rows (only matching customer_ids)')
    print(inner.to_string(index=False))
    print()

    # Left join: keep all customers
    left = pd.merge(customers, orders, on='customer_id', how='left')
    print(f'Left join: {len(left)} rows (all customers, NaN for no orders)')
    print(left.to_string(index=False))
    print()

    # Aggregation after join: total spend per customer
    customer_spend = orders.groupby('customer_id')['amount'].agg(
        total_spend='sum', num_orders='count', avg_order='mean'
    ).reset_index()
    enriched = pd.merge(customers, customer_spend, on='customer_id', how='left')
    enriched = enriched.fillna({'total_spend': 0, 'num_orders': 0, 'avg_order': 0})
    print('=== Enriched Customer Table ===')
    print(enriched.to_string(index=False))
    print()

    # Merge types summary
    merge_ref = pd.DataFrame({
        'Join Type': ['inner', 'left', 'right', 'outer'],
        'Keeps': ['Matching only', 'All left + matching right',
                  'Matching left + all right', 'All from both'],
        'NaN Where': ['Never', 'Right cols for non-matches',
                      'Left cols for non-matches', 'Both sides for non-matches'],
    })
    print('=== Merge Type Reference ===')
    print(merge_ref.to_string(index=False))


demonstrate_merge_operations()

---## Part 5 — Summary & Lessons Learned### Key Takeaways1. **Always start with EDA.** Before any modeling, check data types, missing   values, distributions, and correlations. `df.describe()`, `df.info()`, and   `df.isnull().sum()` are your first three commands on any new dataset.2. **Handle missing values deliberately.** Choose between deletion (if few and   random), imputation (mean/median for numerical, mode for categorical), or   indicator variables (if missingness itself is informative). Never silently   ignore NaN values.3. **Choose encoding based on the feature type.** Use ordinal encoding for   ordered categories (low/medium/high), one-hot for unordered categories with   few values, and target/frequency encoding for high-cardinality features.4. **Feature engineering can matter more than model choice.** Adding log   transforms, interaction features, and domain-specific variables improved   our linear regression more than switching to a fancier algorithm would.5. **Use Pandas for wrangling, NumPy/PyTorch for computation.** Pandas adds   overhead from index management. Convert to NumPy arrays or PyTorch tensors   before training loops.### What's Next→ **01-04 (Visualization with Matplotlib)** teaches the plotting skills needed  to create the EDA visualizations, decision boundaries, and training curves  used throughout this course.### Going Further- [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html) —  Official comprehensive documentation- [Feature Engineering for ML (Alice Zheng)](https://www.oreilly.com/library/view/feature-engineering-for/9781491953235/) —  Practical guide to feature engineering techniques- [Handling Missing Data (scikit-learn)](https://scikit-learn.org/stable/modules/impute.html) —  sklearn's imputation strategies including KNN and iterative imputers